In [9]:
#Importing libraries
from bs4 import BeautifulSoup
import requests
import re
import urllib
import pprint
from urllib import parse
import time
import pickle

In [10]:
#Parent url
url = 'https://www.nhs.uk/servicedirectories/pages/nhstrustlisting.aspx'

In [11]:
#Creating a BeautifulSoup object out of the parent page's HTML
data = requests.get(url)
soup = BeautifulSoup(data.text, 'html.parser')

In [12]:
#Collecting every link on the parent page and assembling it into a dictionary with its name and link as key:value pairs
link_dict = {}
for link in soup.find_all('a'): link_dict.update({str(link.get('title')):str(link.get('href'))})

In [17]:
#Creating a list of all the values in the link dictionary and concatenating it with the parent url to reach the correct end url
link_vals = []

for value in link_dict.values():
    link_vals.append('https://www.nhs.uk/'+value)
print('test')

test


In [14]:
#This is the most complex part of the script

#The list where all the links in each of the linked urls will be stored
firstnestlinks = []

#The regex for matching only links that are contact detail sub-links. This is where the postcodes are found
r = re.compile(".*ContactDetails.")

#Delving into the links' sub-links and adds all of them to the firstnestlinks list
for link in link_vals:
    link = requests.get(link)
    link = BeautifulSoup(link.text,'html.parser')
    link = link.find_all(class_="tabs-nav")
    for tag in link:
        for anchor in tag.find_all('a'):
            firstnestlinks.append(anchor['href'])
            time.sleep(0.15) #This is important! We do not want to DDOS the NHS.

In [15]:
#Finally, taking all the links in the firstnestlinks list and filtering them down to only the Contact Details links
linklist = list(filter(r.match, firstnestlinks))

In [16]:
#Storing the list of links as a pickle object to avoid re-running this script and reduce calls to the NHS' servers
#We will start the next script by calling this object
pickle.dump(linklist, open( "linklist2.p", "wb" ) )